In [ ]:
/storage/ice1/shared/d-pace_community/makerspace-datasets/MEDICAL/OLIVES

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from torch.autograd import Variable

data = np.genfromtxt('/content/Biomarker_Clinical_Data_Images.csv', delimiter=',', skip_header=1, usecols=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20))

with_headers = np.genfromtxt('/content/Biomarker_Clinical_Data_Images.csv', delimiter=',', names=True, usecols=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20))

avgArr = np.zeros([int(data.shape[0]/49), data.shape[1]])

for i in range(0, int(data.shape[0]/49)):
  start = 49*i+1
  scans = data[start:start+48,:]
  avgArr[i] = scans.mean(axis=0)

np.set_printoptions(precision=3, suppress=True)

presence = np.ceil(avgArr)

biomarkerAvg, temp = np.hsplit(avgArr, np.array([16]))
__, scores = np.hsplit(temp, np.array([1]))
biomarkerPresence, __ = np.hsplit(presence, np.array([16]))

avg_train, avg_test, score_train, score_test = train_test_split(biomarkerAvg, scores, train_size = 150, test_size=42, random_state=0)
presence_train, presence_test, __, __ = train_test_split(biomarkerPresence, scores, train_size = 150, test_size=42, random_state=0)

score_train = (score_train - score_train.min(0))/score_train.ptp(0)
score_test = (score_test - score_test.min(0))/score_test.ptp(0)

np.savetxt('presence.csv', presence, delimiter=',')
np.savetxt('average.csv', avgArr, delimiter=',')
np.savetxt('scores.csv', scores, delimiter=',')


In [3]:
torch_avg_train = Variable(torch.from_numpy(avg_train).float())
torch_avg_test = Variable(torch.from_numpy(avg_test).float())
torch_presence_train = Variable(torch.from_numpy(presence_train).float())
torch_presence_test = Variable(torch.from_numpy(presence_test).float())
torch_score_train = Variable(torch.from_numpy(score_train).float())
torch_score_test = Variable(torch.from_numpy(score_test).float())

class FullLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(FullLinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(2, 4)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(4, 16)
    def forward(self, x):
        X = self.linear(x)
        X2 = self.relu(X)
        y_pred = self.linear2(X2)
        return y_pred

In [4]:
class SingleLinearRegressionModel(torch.nn.Module):
    def __init__(self):
        super(SingleLinearRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(2, 1)
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [6]:
try:
  del avgModel
  del presenceModel
  del loss_fn
  del avg_optimizer
  del presence_optimizer
except:
  pass

presenceModel = FullLinearRegressionModel()
avgModel = FullLinearRegressionModel()
loss_fn = torch.nn.MSELoss()
avg_optimizer = torch.optim.SGD(avgModel.parameters(), lr = 0.01)
presence_optimizer = torch.optim.SGD(presenceModel.parameters(), lr = 0.01)

In [9]:
for epoch in range(5000):

    # Forward pass: Compute predicted y by passing
    # x to the model
    pred_y_presence = presenceModel(torch_score_train)
    pred_y_avg = avgModel(torch_score_train)

    # Compute and print loss
    loss_presence = loss_fn(pred_y_presence, torch_presence_train)
    loss_avg = loss_fn(pred_y_avg, torch_avg_train)

    # Zero gradients, perform a backward pass,
    # and update the weights.
    avg_optimizer.zero_grad()
    presence_optimizer.zero_grad()

    loss_presence.backward()
    loss_avg.backward()

    avg_optimizer.step()
    presence_optimizer.step()

    if (epoch % 20 == 0):
      print('epoch {:04}: loss (average):  {}'.format(epoch, loss_avg.item()))
      print('            loss (presence): {}'.format(loss_presence.item()))

test_pred_avg = avgModel(torch_score_test)
test_pred_presence = presenceModel(torch_score_test)
np.set_printoptions(precision=3, suppress=True)
torch.set_printoptions(threshold=10_000)
#print(test_pred_avg)
#print(test_pred_presence)


epoch 0000: loss (average):  0.048526570200920105
            loss (presence): 0.11410733312368393
epoch 0020: loss (average):  0.04851962998509407
            loss (presence): 0.11409852653741837
epoch 0040: loss (average):  0.04851267859339714
            loss (presence): 0.11408958584070206
epoch 0060: loss (average):  0.04850572347640991
            loss (presence): 0.11408066004514694
epoch 0080: loss (average):  0.048498764634132385
            loss (presence): 0.11407177895307541
epoch 0100: loss (average):  0.048491813242435455
            loss (presence): 0.11406290531158447
epoch 0120: loss (average):  0.04848484694957733
            loss (presence): 0.11405405402183533
epoch 0140: loss (average):  0.04847788065671921
            loss (presence): 0.11404523253440857
epoch 0160: loss (average):  0.048470910638570786
            loss (presence): 0.11403641104698181
epoch 0180: loss (average):  0.048463933169841766
            loss (presence): 0.11402761936187744
epoch 0200: los

In [97]:
from sklearn.metrics import accuracy_score, f1_score

for i in range(0,16):
  del singleModel
  biomarker = biomarkerPresence[:,i].reshape(-1,1)

  bio_train, bio_test, __, __ = train_test_split(biomarker, scores, train_size = 150, test_size=42, random_state=0)

  torch_bio_train = Variable(torch.from_numpy(bio_train).float())
  torch_bio_test = Variable(torch.from_numpy(bio_test).float())

  singleModel = SingleLinearRegressionModel()
  loss_fn = torch.nn.MSELoss()
  optimizer = torch.optim.SGD(singleModel.parameters(), lr = 0.01)
  for epoch in range(100):
    pred_y = singleModel(torch_score_train)
    loss = loss_fn(pred_y, torch_bio_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #if (epoch % 50 == 0):
    #  print('epoch {}, loss {}'.format(epoch, loss.item()))
  pred = singleModel(torch_score_test)
  pred = pred > 0.5
  loss = loss_fn(pred, torch_bio_test)
  f1 = f1_score(torch_bio_test, pred, average='binary', zero_division=np.nan)
  #print(f"{with_headers.dtype.names[i]}: {f1}")
  print(f"{i+1}: {f1}")
  #if i == 8:
  #  print(pred)
  #  print(torch_bio_test)
  #print(f"{i+1}: {loss.item()}")

1: 0.0
2: 0.0
3: 0.0
4: 0.0
5: 1.0
6: 0.631578947368421
7: 0.7
8: 0.0
9: 0.935064935064935
10: 0.0
11: 0.6774193548387096
12: 0.9230769230769231
13: 0.3333333333333333
14: nan
15: nan
16: 0.0


In [ ]:
0.75:

1: 0.0
2: 0.0
3: 0.0
4: 0.0
5: 1.0
6: 0.0
7: 0.2857142857142857
8: 0.0
9: 0.8985507246376812
10: 0.0
11: 0.0
12: 0.782608695652174
13: 0.0
14: nan
15: nan
16: 0.0

0.5:

1: 0.0
2: 0.0
3: 0.0
4: 0.125
5: 1.0
6: 0.6551724137931034
7: 0.746268656716418
8: 0.0
9: 0.935064935064935
10: 0.0
11: 0.68
12: 0.9230769230769231
13: 0.0
14: nan
15: nan
16: 0.0


0.25:

1: 0.0
2: 0.4
3: 0.0
4: 0.4444444444444444
5: 1.0
6: 0.746268656716418
7: 0.746268656716418
8: 0.3333333333333333
9: 0.9367088607594937
10: 0.0
11: 0.7076923076923077
12: 0.9230769230769231
13: 0.625
14: nan
15: 0.0
16: 0.6

0.05:

1: 0.21052631578947367
2: 0.38461538461538464
3: 0.2222222222222222
4: 0.4444444444444444
5: 1.0
6: 0.7647058823529411
7: 0.746268656716418
8: 0.41509433962264153
9: 0.95
10: 0.0
11: 0.7076923076923077
12: 0.9230769230769231
13: 0.10810810810810811
14: 0.0
15: 0.0
16: 0.16216216216216217